In [21]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def fetch_listings(search_query=''):
    url = f"https://jp.mercari.com/en/search?keyword=&category_id=1&lang=en&sort=created_time&order=desc&brand_id=1292"
    
    # We use Selenium first to load the page and then pass the source to BeautifulSoup later
    options = webdriver.ChromeOptions()
    options.headless = True 
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    try:
        # Wait until the item grid is present
        wait = WebDriverWait(driver, 20)  # Wait for up to 20 seconds
        item_grid = wait.until(EC.presence_of_element_located((By.ID, 'item-grid')))
        page_source = driver.page_source
    except Exception as e:
        print(f"An error occurred: {e}")
        driver.quit()
        return []
    driver.quit()

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    listings = []
    item_grid = soup.find('div', {'id': 'item-grid'})
    if not item_grid:
        print("Could not find the item grid.")
        return listings

    # Select each item within the grid
    for item in item_grid.select('li[data-testid="item-cell"]'):
        link_tag = item.find('a', {'data-testid': 'thumbnail-link'})
        price_tag = item.find('span', class_='merPrice')
        image_tag = item.find('img', {'loading': 'lazy'})
        
        if link_tag and price_tag:
            link = link_tag['href']
            price = price_tag.text.strip()
            image = image_tag['src']
            listings.append({
                'price': price,
                'link': 'https://jp.mercari.com' + link,
                'image': image
            })

    return listings

def check_for_new_listings():
    new_listings = fetch_listings()

    # Load previous listings
    try:
        with open('previous_listings.json', 'r') as file:
            previous_listings = json.load(file)
    except FileNotFoundError:
        previous_listings = []

    # Compare listings
    new_items = [item for item in new_listings if item not in previous_listings]

    if new_items:
        print("New listings found:")
        for item in new_items:
            print(item)
        # Here you can add code to send notifications
    else:
        print("No new listings.")

    # Save current listings for future comparison
    with open('previous_listings.json', 'w') as file:
        json.dump(new_listings, file)

check_for_new_listings()


No new listings.
